In [ ]:
import pandas as pd
import numpy as np
import data_handler
from tqdm import tqdm
import matplotlib.pyplot as plt

from torchvision import datasets
import os
from itertools import chain, product

from scipy.stats import ttest_1samp, ttest_rel, t
from scipy import stats

In [ ]:
DATA_PATH = os.path.join(os.path.curdir, "data", "survey_results", "data_tu-helpfulness-of-xai_2022-07-13_13-03.xlsx")
DATA_PREPARED_PATH = f"{os.path.splitext(DATA_PATH)[0]}_PREPARED{os.path.splitext(DATA_PATH)[1]}"
df = pd.read_excel(DATA_PATH)
df_merged = pd.read_excel(DATA_PREPARED_PATH)

In [ ]:
META_DATA_PATH = os.path.join(os.path.curdir, "data", "survey_results", "question_meta_data.xlsx")
df_quest_meta = pd.read_excel(META_DATA_PATH)

# Evaluation Metrics Calculation & Testing

### Metric functions

In [ ]:
def accuracy_per_groupby(df_merged, group_by=["method"], drop_na=False):
    # accuracy = (TP + TN) / (TP + TN + FP + FN)
    schema_cols = [df_merged[column].drop_duplicates() for column in group_by + ["is_pred_correct", "response"]]
    schema_col_names = [ser.name for ser in schema_cols]
    df_schema = pd.DataFrame(list(product(*schema_cols))).groupby(list(np.arange(len(schema_cols)))).count()
    df_schema = df_schema.reset_index()
    df_schema.columns=schema_col_names
    
    ser_denominators = df_merged.groupby(by=group_by, dropna=drop_na)["response"].count().sort_index()
    df_numerators = df_merged.groupby(by=group_by + ["is_pred_correct", "response"], dropna=drop_na)["response"].count().to_frame("count").reset_index()
    df_numerators = df_schema.merge(right=df_numerators, on=schema_col_names, how="left").fillna(0)
    ser_numerators = df_numerators[df_numerators["is_pred_correct"] == df_numerators["response"]].groupby(group_by, dropna=drop_na)["count"].sum().sort_index()
    df_accuracies = (ser_numerators / ser_denominators).to_frame("accuracy")
    return df_accuracies

In [ ]:
def sensitivity_per_groupby(df_merged, group_by=["method"], drop_na=True):
    # sensitivity = TP / (TP + FN)
    schema_cols = [df_merged[column].drop_duplicates() for column in group_by + ["is_pred_correct", "response"]]
    schema_col_names = [ser.name for ser in schema_cols]
    df_schema = pd.DataFrame(list(product(*schema_cols))).groupby(list(np.arange(len(schema_cols)))).count()
    df_schema = df_schema.reset_index()
    df_schema.columns=schema_col_names
    
    df_merged_filt = df_merged[(df_merged["is_pred_correct"] == True)]
    ser_denominators = df_merged_filt.groupby(by=group_by, dropna=drop_na)["response"].count().sort_index()
    df_numerators = df_merged_filt.groupby(by=group_by + ["is_pred_correct", "response"], dropna=drop_na)["response"].count().to_frame("count").reset_index()
    df_numerators = df_schema.merge(right=df_numerators, on=schema_col_names, how="left").fillna(0)
    ser_numerators = df_numerators[df_numerators["is_pred_correct"] == df_numerators["response"]].groupby(group_by, dropna=drop_na)["count"].sum().sort_index()
    df_sensitivity = (ser_numerators / ser_denominators).to_frame("sensitivity")
    return df_sensitivity

In [ ]:
def specificity_per_groupby(df_merged, group_by=["method"], drop_na=True):
    # specificity = TN / (TN + FP)
    schema_cols = [df_merged[column].drop_duplicates() for column in group_by + ["is_pred_correct", "response"]]
    schema_col_names = [ser.name for ser in schema_cols]
    df_schema = pd.DataFrame(list(product(*schema_cols))).groupby(list(np.arange(len(schema_cols)))).count()
    df_schema = df_schema.reset_index()
    df_schema.columns=schema_col_names
    
    df_merged_filt = df_merged[(df_merged["is_pred_correct"] == False)]
    ser_denominators = df_merged_filt.groupby(by=group_by, dropna=drop_na)["response"].count().sort_index()
    df_numerators = df_merged_filt.groupby(by=group_by + ["is_pred_correct", "response"], dropna=drop_na)["response"].count().to_frame("count").reset_index()
    df_numerators = df_schema.merge(right=df_numerators, on=schema_col_names, how="left").fillna(0)
    ser_numerators = df_numerators[df_numerators["is_pred_correct"] == df_numerators["response"]].groupby(group_by, dropna=drop_na)["count"].sum().sort_index()
    df_specificity = (ser_numerators / ser_denominators).to_frame("specificity")
    return df_specificity

In [ ]:
def get_metric_convergence(df_metric):
    metric_name = df_metric.columns[0]
    return df_metric.groupby(level=0).expanding().mean().reset_index(level=[1, 2]).\
            pivot(index="case", columns="method", values=metric_name)

### Plotting functions

In [ ]:
def plot_metric_boxplot(df_metrics, by, column, xlabel="XAI-Method", ylabel="Accuracy", title=""):
    ax = df_metrics.boxplot(by=by, column=column, figsize=(14, 5), showmeans=True, meanline=True)
    fig = plt.gcf()
    fig.suptitle(None)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    #xai = df_metrics[]
    #ax.plot(["ConfidenceScores", "gradCAM"], [0.5, 0.5], c="red", linestyle="dashed")
    plt.show()
    plt.close()

In [ ]:
def plot_sensitivity_specificity(df_sens_mean, df_spec_mean, title_addition=""):
    df_sens_mean = df_sens_mean.sort_index()
    df_spec_mean = df_spec_mean.sort_index()
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.scatter(x=df_sens_mean.index, y=df_sens_mean, c="darkorange", label="Sensitivity: Guessed Correct when Model Correct")
    ax.scatter(x=df_spec_mean.index, y=df_spec_mean, c="olivedrab", label="Specificity: Guessed Wrong when Model Wrong", marker="^")
    ax.plot(df_sens_mean["mean"].index, [0.5] * len(df_sens_mean), color="indianred", linestyle='dashed', label="baseline")
    ax.set_title(f"{title_addition} Performance Ratios for chosen XAI-Methods ({len(df)-1} participants considered)", size=15)
    ax.set_xlabel("XAI-Method", size=13)
    ax.set_ylabel("Ratio", size=13)
    for i, txt in enumerate(df_sens_mean["mean"].round(2)):
        ax.annotate(txt, (list(df_sens_mean.index)[i], list(df_sens_mean["mean"])[i]))
    for i, txt in enumerate(df_spec_mean["mean"].round(2)):
        ax.annotate(txt, (list(df_spec_mean.index)[i], list(df_spec_mean["mean"])[i]))
    ax.legend(loc="best", edgecolor="black")
    ax.tick_params(labelright=True)
    ax.yaxis.set_ticks_position('both')
    plt.savefig("ratios.svg") # sens_vs_spec.svg

In [ ]:
def plot_metric_convergence(df_metrics_conv, metric_name="Accuracy"):
    methods_sorted = df_metrics_conv.iloc[-1].sort_values(ascending=False).index.values
    df_metrics_conv = df_metrics_conv[methods_sorted]
    
    methods_names = methods_sorted.copy()
    methods_names.sort()
    colors =["darkorange", "olivedrab","lightseagreen","darkslategrey","royalblue","mediumvioletred"]
    line = ["dotted","dashed","dashdot","dotted","dashed","dashdot"]
    colors_dict = dict(zip(methods_names, colors))
    line_dict = dict(zip(methods_sorted, line))
    
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.set_xlabel("Number Participants", size=13)
    ax.set_ylabel("Accuracy", size=13)
    #ax.set_title(f"Convergence of {metric_name} as Number of Participants increase", size=15)
    df_metrics_conv.apply(lambda method_conv: ax.plot(list(range(len(method_conv))), method_conv, label=method_conv.name, color=colors_dict[method_conv.name],ls=line_dict[method_conv.name]))
    ax.legend(loc="upper right", edgecolor="black")
    ax.tick_params(labelright=True)
    ax.yaxis.set_ticks_position('both')
    plt.savefig("convergence.pdf")

In [ ]:
def plot_2_indexes_1_plot(df_double_index, xlabel, ylabel, title, drop_na=True, drop_duplicates=True, mean_idx_order=None):
    fig, ax = plt.subplots(2, 1, figsize=(20, 10))
    plt.setp(ax, ylim=(0, 1.1))
    if drop_duplicates:
        ser_level_1_count = df_merged.drop_duplicates("case").groupby(df_double_index.index.levels[0].name, dropna=drop_na)[df_merged.columns[0]].count()
    else:
        ser_level_1_count = df_merged.groupby(df_double_index.index.levels[0].name, dropna=drop_na)[df_merged.columns[0]].count()
                                                                                                                            
    ser_acc_mean = df_double_index.mean(level=0)[df_double_index.columns[0]]
    # reorder bars in barplot as specified
    if mean_idx_order:
        ser_acc_mean = ser_acc_mean.reindex(mean_idx_order)
                                                                                                                         
    pps = ax[0].bar(ser_acc_mean.index, ser_acc_mean)
    for p in pps:
        height = p.get_height()
        ax[0].text(x=p.get_x() + p.get_width() / 2, y=height+.05,
          s=round(height, 2),
          ha='center', size=12)
    ax[0].set_xlabel(ser_acc_mean.index.name, size=15)
    ax[0].set_ylabel("Accuracy", size=15)
    ax[0].set_title(f"Mean Accuracy over all XAI-Methods per {ser_acc_mean.index.name}", size=20)
    
    for idx_level_1 in df_double_index.index.levels[0]:
        ax[1].plot(df_double_index.loc[idx_level_1].index, list(df_double_index.loc[idx_level_1][df_double_index.columns[0]]), label=f"{idx_level_1} ({ser_level_1_count[idx_level_1]})")
        ax[1].scatter(df_double_index.loc[idx_level_1].index, list(df_double_index.loc[idx_level_1][df_double_index.columns[0]]))
    ax[1].legend()
    ax[1].set_xlabel(xlabel, size=15)
    ax[1].set_ylabel(ylabel, size=15)
    ax[1].set_title(title, size=20)
    plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.3, wspace=0.4)
    plt.show()
    plt.close()

In [ ]:
def bar_plot(df_double_index,filename ,drop_na=True, drop_duplicates=True, mean_idx_order=None):
    fig, ax = plt.subplots(figsize=(14, 5))
    if drop_duplicates:
        ser_level_1_count = df_merged.drop_duplicates("case").groupby(df_double_index.index.levels[0].name, dropna=drop_na)[df_merged.columns[0]].count()
    else:
        ser_level_1_count = df_merged.groupby(df_double_index.index.levels[0].name, dropna=drop_na)[df_merged.columns[0]].count()

    ser_acc_mean = df_double_index.mean(level=0)[df_double_index.columns[0]]
    # reorder bars in barplot as specified
    if mean_idx_order:
        ser_acc_mean = ser_acc_mean.reindex(mean_idx_order)

    pps = ax.bar(ser_acc_mean.index, ser_acc_mean,color="olivedrab")
    for p in pps:
        height = p.get_height()
        ax.text(x=p.get_x() + p.get_width() / 2, y=height+.05,
          s=round(height, 2),
          ha='center', size=12)
    ax.set_xlabel(ser_acc_mean.index.name, size=15)
    ax.set_ylabel("Accuracy", size=15)
    #ax.set_title(f"Mean Accuracy over all XAI-Methods per {ser_acc_mean.index.name}", size=20)
    ax.tick_params(labelright=True)
    ax.set_ylim([0, 1])
    plt.savefig(filename+".svg")
    plt.show()
    plt.close()

In [ ]:
def line_plot(df_double_index, xlabel, ylabel,filename, drop_na=True):
    fig, ax = plt.subplots(figsize=(14, 5))

    colors =["darkorange","olivedrab"]
    line = ["dashed","dashdot"]
    idx_level_1 = df_double_index.index.levels[0]

    for i in range(idx_level_1.shape[0]):
        ax.plot(df_double_index.loc[idx_level_1[i]].index, list(df_double_index.loc[idx_level_1[i]][df_double_index.columns[0]]),
                label=f"{idx_level_1[i]}",color=colors[i],ls=line[i]) #({ser_level_1_count[idx_level_1]})
        ax.scatter(df_double_index.loc[idx_level_1[i]].index, list(df_double_index.loc[idx_level_1[i]][df_double_index.columns[0]]),
                   color=colors[i])
    ax.legend(loc="upper right")
    ax.set_xlabel(xlabel, size=13)
    ax.set_ylabel(ylabel, size=13)
    ax.set_ylim([0, 1])
    ax.tick_params(labelright=True)
    plt.savefig(filename+".svg")
    plt.show()
    plt.close()

### Testing & effect size functions

In [ ]:
# isn't used currently
def print_test_result(pval, alpha, mean_h_0):
    print(f"p-value: {pval}")
    if pval < alpha:
        print(f"Reject H0: accuracy_mean <= {mean_h_0} \n")
    else:
        print("Can't reject H0 \n")

In [ ]:
def perform_t_test(accuracies, mean_h_0, alpha, alternative):
    tset, pval = ttest_1samp(a=accuracies, popmean=mean_h_0, alternative=alternative)
    print(accuracies.name)
    print(f"p-value: {pval}")
    if pval < alpha:
        print(f"Reject H0: accuracy_mean <= {mean_h_0} \n")
    else:
        print("Can't reject H0 \n")
    return tset

In [ ]:
def perform_t_test_with_flexible_alternative(metrics, mean_h_0, alpha):
    h_a = None
    sample_mean = metrics.mean()
    if sample_mean < mean_h_0:
        h_a = "less"
    elif sample_mean > mean_h_0:
        h_a = "greater"
    else:
        h_a = "two-sided"
    tset, pval = ttest_1samp(a=metrics, popmean=mean_h_0, alternative=h_a)
    
    h_a_to_h_0_mapping = {"less": ">=", "greater": "<=", "two-sided": "equal"}
    print(f"{metrics.name}:  sample_mean={sample_mean}")
    print(f"H0: sample_mean {h_a_to_h_0_mapping[h_a]} {mean_h_0}")
    print(f"p-value: {pval}")
    if pval < alpha:
        print(f"Reject H0! (alpha={alpha})\n")
    else:
        print(f"Can't reject H0: (alpha={alpha})\n")
    return tset

In [ ]:
def get_pairwise_paired_t_test_alternatives_and_p_values(df_metrics, df_metrics_mean):
    paired_test_alternatives = []
    paired_test_pvalues = []
    df_metrics_mean = df_metrics_mean.sort_values(by="mean", ascending=False)
    xai_accuracy_sorted = df_metrics_mean.index.values
    for method_i in xai_accuracy_sorted:
        method_i_alternatives = []
        method_i_pvalues = []
        for method_j in xai_accuracy_sorted:
            # h_a = "less" if df_metrics_mean.loc[method_i]["mean"] < df_metrics_mean.loc[method_j]["mean"] else "greater" if df_metrics_mean.loc[method_i]["mean"] > df_metrics_mean.loc[method_j]["mean"] else None
            h_a = None
            if df_metrics_mean.loc[method_i]["mean"] < df_metrics_mean.loc[method_j]["mean"]:
                h_a = "less"
            elif df_metrics_mean.loc[method_i]["mean"] > df_metrics_mean.loc[method_j]["mean"]:
                h_a = "greater"
            elif (df_metrics_mean.loc[method_i]["mean"] == df_metrics_mean.loc[method_j]["mean"]) & (method_i != method_j):
                h_a = "two-sided"
            method_i_alternatives.append(h_a)
            if h_a is None:
                method_i_pvalues.append(np.nan)
            else:
                a = df_metrics[method_i].values
                b = df_metrics[method_j].values
                tset, pval = ttest_rel(a, b, alternative=h_a)
                method_i_pvalues.append(pval)
        paired_test_alternatives.append(method_i_alternatives)
        paired_test_pvalues.append(method_i_pvalues)
        
    df_paired_alternatives = pd.DataFrame(paired_test_alternatives, index=xai_accuracy_sorted, columns=xai_accuracy_sorted)
    df_paired_alternatives.index.name = "method_i"
    df_paired_alternatives.columns.name = "method_j"
    
    df_paired_pvalues = pd.DataFrame(paired_test_pvalues, index=xai_accuracy_sorted ,columns=xai_accuracy_sorted)
    df_paired_pvalues.index.name = "method_i"
    df_paired_pvalues.columns.name = "method_j"
    
    return df_paired_alternatives, df_paired_pvalues

In [ ]:
# erste Formel
def cohens_d(mean_1, var_1, mean_2, var_2):
    return (mean_1 - mean_2) / np.sqrt((var_1 + var_2) / 2)

In [ ]:
# Cohen’s d for one-sample t-test
# https://www.youtube.com/watch?v=yGc1a324t14&ab_channel=Dr.ToddGrande (1-sample t-test)
# https://www.youtube.com/watch?v=yVbYvn_cT5w&ab_channel=Dr.ToddGrande (2-sample t-test)
# noch andere Quellen??? --> keine gefunden!
def cohens_d_(t_statistic, N):
    return t_statistic / np.sqrt(N)

### Constants

In [ ]:
# significance level
ALPHA = 0.05

## 1. Accuracies

In [ ]:
df_acc = accuracy_per_groupby(df_merged, group_by=["method", "case"])
df_acc_pivot = df_acc.reset_index().pivot(index="case", columns="method", values="accuracy")
df_acc_frequencies = df_acc_pivot.apply(lambda x: x.value_counts())
df_acc_conv = get_metric_convergence(df_acc)

### Accuracy means

In [ ]:
df_acc_mean = df_acc_pivot.mean().to_frame("mean").sort_values(by="mean", ascending=False)
df_acc_mean.round(2)

In [ ]:
# overall
df_acc_mean.mean().to_frame("accuracy").mean()

### Accuracy boxplots

In [ ]:
plot_metric_boxplot(df_acc.reset_index(), ["method"], "accuracy", title="Accuracy Variance among Participants per XAI-Method")

### Accuracy convergence

In [ ]:
plot_metric_convergence(df_acc_conv)

### Accuracy values  frequencies (all possible discrete according to question resolution)

In [ ]:
ax = df_acc_frequencies[df_acc_mean.index].transpose().plot.bar(figsize=(13, 8))
ax.set_ylabel("Number Participants who reached accuracy score")

### Significance & Effect Size: Accuracy comparisons to random baseline

#### 1-sample t-test

In [ ]:
mean_h_0 = 0.5
alternative_acc = "greater"

In [ ]:
ser_acc_1_sample_t_values = df_acc_pivot.apply(lambda accuracies_method: perform_t_test(accuracies_method, mean_h_0, ALPHA, alternative_acc))

In [ ]:
# test with flexibile alternative
df_acc_pivot.apply(lambda accuracies_method: perform_t_test_with_flexible_alternative(accuracies_method, mean_h_0, ALPHA))

#### Effect size (Cohens d) 

In [ ]:
mean_random_guess = 0.5
variance_random_guess = 0.0625 # deriviation via binomial distribution of correct answers

In [ ]:
df_acc_var = df_acc_pivot.var().to_frame("var")
df_cohens_d = pd.concat([df_acc_mean.T, df_acc_var.T])
df_random_mean_var = pd.DataFrame({"mean_random" : [mean_random_guess]*6, "var_random": [variance_random_guess]*6}, index=df_cohens_d.columns).T
df_cohens_d = pd.concat([df_cohens_d, df_random_mean_var])


In [ ]:
# In welcher Reihenfolge müssen werte cohends_d Formel übergegebn werden, damit es wie interpretiert werden kann?
df_cohens_d.apply(lambda x: cohens_d(*x))

In [ ]:
# nochmal mit Formel für 
N_acc = df_acc_pivot.shape[0]
df_acc_1_sample_effect = ser_acc_1_sample_t_values.map(lambda t_val: cohens_d_(t_val, N_acc)).to_frame("cohens_d").sort_values(by="cohens_d", ascending=False)

In [ ]:
df_acc_1_sample_effect

### Significance & Effect Size: Accuracies pairwise between each other

#### Paired t-test

In [ ]:
df_acc_paired_test_alternatives, df_acc_apaired_test_pvalues = get_pairwise_paired_t_test_alternatives_and_p_values(df_acc_pivot, df_acc_mean)

In [ ]:
df_acc_paired_test_alternatives

In [ ]:
df_acc_apaired_test_pvalues

In [ ]:
df_acc_apaired_test_pvalues < ALPHA

In [ ]:
# Test Results

#### Effect size in compare (Cohens d)

## 2. Sensitivity

In [ ]:
df_sens = sensitivity_per_groupby(df_merged, group_by=["method", "case"])
df_sens_pivot = df_sens.reset_index().pivot(index="case", columns="method", values="sensitivity")
df_sens_frequencies = df_sens_pivot.apply(lambda x: x.value_counts())
df_sens_conv = get_metric_convergence(df_sens)

### Sensitivity means

In [ ]:
df_sens_mean = df_sens_pivot.mean().to_frame("mean").sort_values(by="mean", ascending=False)
df_sens_mean

In [ ]:
# overall
df_sens_mean.mean()

### Sensitivity boxplots

In [ ]:
plot_metric_boxplot(df_sens.reset_index(), ["method"], "sensitivity", title="Sensitivity Variance among Participants per XAI-Method")

### Sensitvity convergence

In [ ]:
plot_metric_convergence(df_sens_conv, metric_name="Sensitivity")

### Sensitivity values  frequencies (all possible discrete according to question resolution)

In [ ]:
ax = df_sens_frequencies[df_sens_mean.index].transpose().plot.bar(figsize=(13, 8))
ax.set_ylabel("Number Participants who reached accuracy score")

### Significance & Effect Size: Sensitivity comparisons to random baseline

#### 1-sample t-test

In [ ]:
mean_h_0 = 0.5
alternative_sens = "greater"

In [ ]:
ser_sens_1_sample_t_values = df_sens_pivot.apply(lambda metric_method: perform_t_test(metric_method, mean_h_0, ALPHA, alternative_sens))


In [ ]:
# test with flexibile alternative
df_sens_pivot.apply(lambda metric_method: perform_t_test_with_flexible_alternative(metric_method, mean_h_0, ALPHA))

#### Effect size (Cohens d) 

### Significance & Effect Size: Sensitivity pairwise between in each other

#### Paired t-test

In [ ]:
df_sens_paired_test_alternatives, df_sens_apaired_test_pvalues = get_pairwise_paired_t_test_alternatives_and_p_values(df_sens_pivot, df_sens_mean)


In [ ]:
df_sens_paired_test_alternatives

In [ ]:
df_sens_apaired_test_pvalues

In [ ]:
df_sens_apaired_test_pvalues < ALPHA

In [ ]:
# Test Results for Sensitivity (rowise and only the upper diagonal because symmetric)

# gradCAM's sensitivity isnt't significantly greater than CS's and LRP's but bigger than of LIME, IG and SHAP
# CS's sensitivity isnt't significantly greater than LRP's but bigger than of LIME, IG and SHAP
# LRP's sensitivity is significantly greater than LIME's, IG's and SHAP's.
# LIME's sensitivity isn't significantly greater than IG's but bigger than SHAP's
# IG's sensitivity is significantly greater than SHAPS's.
# (SHAPS sensitivity is significantly smaller than all others) --> this is an implication of the above mentioned

#### Effect size in compare (Cohens d)

## 3. Specificity

In [ ]:
df_spec = specificity_per_groupby(df_merged, group_by=["method", "case"])
df_spec_pivot = df_spec.reset_index().pivot(index="case", columns="method", values="specificity")
df_spec_frequencies = df_spec_pivot.apply(lambda x: x.value_counts())
df_spec_conv = get_metric_convergence(df_spec)

### Specificity means

In [ ]:
df_spec_mean = df_spec_pivot.mean().to_frame("mean").sort_values(by="mean", ascending=False)
df_spec_mean

In [ ]:
# overall
df_spec_mean.mean().to_frame("accuracy").mean()

### Specificity boxplots

In [ ]:
plot_metric_boxplot(df_spec.reset_index(), ["method"], "specificity", title="Specificity Variance among Participants per XAI-Method")

### Specificity convergence

In [ ]:
plot_metric_convergence(df_spec_conv, metric_name="Specificity")

### Specificity values frequencies (all possible discrete according to question resolution)

In [ ]:
ax = df_sens_frequencies[df_sens_mean.index].transpose().plot.bar(figsize=(13, 8))
ax.set_ylabel("Number Participants who reached accuracy score")

### Significance & Effect Size: Specificity comparisons to random baseline

#### 1-sample t-test

In [ ]:
mean_h_0 = 0.5
alternative_spec = "greater"

In [ ]:
ser_spec_1_sample_t_values = df_spec_pivot.apply(lambda metric_method: perform_t_test(metric_method, mean_h_0, ALPHA, alternative_spec))


In [ ]:
# test with flexibile alternative
df_spec_pivot.apply(lambda metric_method: perform_t_test_with_flexible_alternative(metric_method, mean_h_0, ALPHA))

#### Effect size (Cohens d) 

### Significance & Effect Size: Specificity pairwise between in each other

#### Paired t-test

In [ ]:
df_spec_paired_test_alternatives, df_spec_apaired_test_pvalues = get_pairwise_paired_t_test_alternatives_and_p_values(df_spec_pivot, df_spec_mean)


In [ ]:
df_spec_paired_test_alternatives

In [ ]:
df_spec_apaired_test_pvalues

In [ ]:
df_spec_apaired_test_pvalues < ALPHA

In [ ]:
# Test Results for Specificity (rowise and only the upper diagonal because symmetric)

# SHAPS's specificity is significantly greater than of all other methods
# CS's specificity is significantly greater than of IG, LIME, gradCAM, LRP
# IG's specificity isn't significantly greater LIME's but greater than gradCAM's and LRP's.
# LIME's specifcity is significantly greater than gradCAM's and LRP's
# gradCAM's specifcity isn't significantly greater than LRP's

In [ ]:
df_spec_mean

#### Effect size in compare (Cohens d)

## 4. Sensitivity vs. Specificity per method

In [ ]:
plot_sensitivity_specificity(df_sens_mean, df_spec_mean, title_addition="")

#### Sensitvity/ Specificity only on fixed images

In [ ]:
fixed_img_idxs = df_quest_meta["img_idx"].value_counts()[df_quest_meta["img_idx"].value_counts() == 12].index
df_quest_meta_fixed = df_quest_meta[df_quest_meta["img_idx"].isin(fixed_img_idxs)]

## 5. Analysis: Participants characteristics on accuracy

### Accuracy for different education levels

In [ ]:
df_accuracy_edu_method = accuracy_per_groupby(df_merged, group_by=["Education", "method"])

In [ ]:
plot_2_indexes_1_plot(df_accuracy_edu_method, "XAI Method", "Accuracy", "Accuracy Scores for different XAI Methods for different Education Levels")

In [ ]:
bar_plot(df_accuracy_edu_method,"education")

### Accuracy for different ML-Experience

In [ ]:
df_accuracy_mlex_method = accuracy_per_groupby(df_merged, group_by=["ML Experience", "method"])

In [ ]:
plot_2_indexes_1_plot(df_accuracy_mlex_method, "XAI Method", "Accuracy", "Comparing Accuracy Scores for different ML Experience Length", mean_idx_order=["Not at all", "Under 1 year", "Between 1 and 3 years", "Between 3 and 5 years", "More than 5 years"])

In [ ]:
bar_plot(df_accuracy_mlex_method,"ml_exp")

### Accuracy for different perceived ML-Experience Usefulness

In [ ]:
df_accuracy_xaiusef_method = accuracy_per_groupby(df_merged, group_by=["ML Experience Usefulness", "method"])

In [ ]:
plot_2_indexes_1_plot(df_accuracy_xaiusef_method, "XAI Method", "Accuracy", "Comparing Accuracy Scores for different perceived ML-Experience Usefulness")

In [ ]:
bar_plot(df_accuracy_xaiusef_method,"ml_exp_self")

### Accuracy for visual impairment

In [ ]:
df_accuracy_visimp_method = accuracy_per_groupby(df_merged, group_by=["Visual Impairment Affect", "method"], drop_na=True)

In [ ]:
plot_2_indexes_1_plot(df_accuracy_visimp_method, "XAI Method", "Accuracy", "Comparing Accuracy Scores for different Visual Impairment Affect", drop_duplicates=True)

In [ ]:
bar_plot(df_accuracy_visimp_method,"visual")

### Accuracy for colorblindness yes/no

In [ ]:
df_accuracy_colorb_method = accuracy_per_groupby(df_merged, group_by=["Color Blindness", "method"])

In [ ]:
plot_2_indexes_1_plot(df_accuracy_colorb_method, "XAI Method", "Accuracy", "Comparing Accuracy Scores for different Color Blindness", drop_duplicates=True)

## 6. Accuracy for different models

In [ ]:
df_accuracy_model_method = accuracy_per_groupby(df_merged, group_by=["model", "method"])

In [ ]:
plot_2_indexes_1_plot(df_accuracy_model_method, "XAI Method", "Accuracy", "Comparing Accuracy Scores for different Models", drop_duplicates=False)

In [ ]:
line_plot(df_accuracy_model_method, "XAI Method", "Accuracy","models")

## 7. Metrics per questionnaire form (detect outlier questionnaires)

In [ ]:
df_accuracy_qustnr_method = accuracy_per_groupby(df_merged, group_by=["QUESTNNR", "method"])

In [ ]:
plot_2_indexes_1_plot(df_accuracy_qustnr_method, "XAI Method", "Accuracy", "Comparing Accuracy Scores for different Questionnaire Forms")

#### Accuracy confidence intervalls per method 

In [ ]:
loc = accuracy_per_groupby(df_merged, group_by=["method", "case"]).mean(level=0),
N = accuracy_per_groupby(df_merged, group_by=["method", "case"]).count(level=0)
std = accuracy_per_groupby(df_merged, group_by=["method", "case"]).std(level=0)
degf = N - 1

low_bounds, upp_bounds = stats.t.interval(0.95, degf,loc=loc, scale=std/np.sqrt(N))

In [ ]:
low_bounds

In [ ]:
upp_bounds